In [2]:
import os

In [3]:
%pwd

'c:\\Users\\sahil\\OneDrive\\Desktop\\Projects\\AI Projects\\Text-Summarizer-Application\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\sahil\\OneDrive\\Desktop\\Projects\\AI Projects\\Text-Summarizer-Application'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [11]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:

        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\sahil\OneDrive\Desktop\Projects\AI Projects\Text-Summarizer-Application\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def tokenize_content(self,data):
        dialogues = data['dialogue']
        summaries = data['summary']

        inputs = ["summarize: " + d if d else "summarize: " for d in dialogues]
        targets = [s if s else "" for s in summaries]

        # Tokenize WITHOUT padding="max_length"
        input_encodings = self.tokenizer(
            inputs,
            max_length=1024,
            truncation=True
        )

        target_encodings = self.tokenizer(
            text_target=targets,
            max_length=128,
            truncation=True
        )

        # Just return the raw lists. No manual -100 replacement needed!
        return {
            "input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]
        }

    def tokenize(self):
        logger.info("Loading dataset from disk...")
        dataset = load_from_disk(self.config.data_path)

        logger.info("Tokenizing the dataset...")
        tokenized_dataset = dataset.map(
            self.tokenize_content,
            batched=True
        )

        logger.info("Saving tokenized dataset to disk...")
        tokenized_dataset.save_to_disk(self.config.root_dir)

In [16]:
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.tokenize()
except Exception as e:
    logger.exception(f"An error occurred during data transformation: {e}")

[2026-02-22 20:51:59,485: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-22 20:51:59,486: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-22 20:51:59,487: INFO: common: Directory created at: artifacts]
[2026-02-22 20:51:59,488: INFO: common: Directory created at: artifacts/data_transformation]
[2026-02-22 20:52:00,274: INFO: 1282601167: Loading dataset from disk...]
[2026-02-22 20:52:00,283: INFO: 1282601167: Tokenizing the dataset...]


Map: 100%|██████████| 818/818 [00:00<00:00, 9781.64 examples/s]

[2026-02-22 20:52:01,916: INFO: 1282601167: Saving tokenized dataset to disk...]



Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 172037.34 examples/s]
